### 1. load dataset

In [1]:
from glob import glob
import json
import gzip, pickle
import re
from typing import List, Tuple

train_dataset_pattern = "../data/train.pickle"
valid_dataset_pattern = "../data/validation.pickle"

In [2]:
with gzip.open(train_dataset_pattern, 'rb') as f:
    train_data = pickle.load(f)

train_data.head(5)

,source,ids,passage,annotation
0,report,REPORT-minute-00001-00004,"위원장 신학용] ""수고하셨습니다. 다음은 존경하는 현영희 위원님 인사말씀해 주세...",{'summary1': '아이들의 미래를 열어가는 장을 만들 수 있는 교과위에 오기...
1,report,REPORT-minute-00001-00006,"위원장 신학용] ""수고하셨고요. 이상민 위원님 인사말씀해 주세요.""\n이상민 위원...",{'summary1': '이 위원은 교육과 과학이 국가의 미래이고 서민들의 삶을 결...
2,report,REPORT-minute-00001-00019,"교육과학기술부장관 이주호] ""96년부터 시행을 하고 있습니다.""\n강은희 위원] ""...",{'summary1': '강 위원은 진보 성향의 교육감을 둔 지역은 공교롭게도 20...
3,report,REPORT-minute-00001-00032,"교육과학기술부장관 이주호] ""저희도 그 부분이 굉장히 가장 중요한 부분이라고 생각하...",{'summary1': '이 교육과학기술부장관은 주 5일제가 잘 정착하기 위해 학교...
4,report,REPORT-minute-00001-00033,"교육과학기술부장관 이주호] ""예.""\n김상희 위원] ""결정을 해서 지금 이제 권고를...",{'summary1': '김 위원은 야당 국회의원인 시인 도 씨의 작품을 교과서에서...


In [3]:
with gzip.open(valid_dataset_pattern, 'rb') as f:
    val_data = pickle.load(f)

val_data.head(5)

,source,ids,passage,annotation
0,report,REPORT-minute-00001-00021,"강은희 위원] ""마지막으로 한 가지 더 여쭙겠습니다. 최근 민간자격증 관해서도 ...",{'summary1': '강 의원은 민간자격증 논란과 관련해 교육부 차원에서 민간자...
1,report,REPORT-minute-00001-00034,"교육과학기술부장관 이주호] ""제가 앞에서도 말씀드렸지만 그게 위탁업무이기 때문에 그...",{'summary1': '교육과학기술부장관 이 씨는 위탁업무이기 때문이라고 말하자 ...
2,report,REPORT-minute-00001-00045,"교육과학기술부장관 이주호] ""예.""\n박혜자 위원] ""그 이유는 지역과 학교의 다양...",{'summary1': '박 위원은 교육부를 해체시켜 연구학습부로 개편하자고 한 장...
3,report,REPORT-minute-00001-00059,"교육과학기술부장관 이주호] ""예 어떻든 하여튼 최선 다해서……""\n이상민 위원] ""...",{'summary1': '컨트롤타워가 누군지 묻는 이 위원의 말에 이 장관은 국과위...
4,report,REPORT-minute-00001-00060,"이상민 위원] ""아니 보세요. 장관님 예산에 대한 조정ㆍ배분권만 갖고 있으면 대한민...",{'summary1': '이 위원은 이 정부의 최대 국정 파탄의 예는 과학기술에 대...


In [4]:
# pickle load 후 ids, dialogues, summaries로 분리
def load_pickle_data(path: str) -> Tuple[List[str], List[List[str]], List[str]]:

    with gzip.open(path, "rb") as f:
        data = pickle.load(f)

    ids, dialogues, summaries = [], [], []

    ids.extend(data["ids"])

    for idx in range(len(data)):
        passage = data["passage"][idx]

        if data["source"][idx] == "dacon":  # dacon
            passage = re.sub(r"\([^)]*\)", "", passage)  # 괄호문 제거
            dialogues.append(passage.split("화자]"))
        elif "speech" in data["ids"][idx]:  # report-speech
            passage_split = re.split(r"\([^)]*\)|\<[^>]*\>", passage)  # 화자와 발화문 분리 "(화자) , <화자> 제거"
            splits = []
            for i in range(len(passage_split)):
                sentence = passage_split[i].strip(" ")  # 양 쪽 공백 제거
                if sentence != "":  # 빈 문자열 무시
                    splits.append(sentence)
            dialogues.append(splits)
        else:  # report-minute, broadcast
            passage_split = re.split(r"\n", passage)  # 발화문별 분리
            splits = []
            for i in range(len(passage_split)):
                sentence = passage_split[i]
                sentence = re.sub(r"^.*]", "", sentence)  # 화자] 제거
                sentence = sentence.strip('" ')
                if sentence != "":  # 빈 문자열 무시
                    splits.append(sentence)
            dialogues.append(splits)

    summaries.extend([summary["summary1"] for summary in data["annotation"]])

    return ids, dialogues, summaries

In [5]:
train_ids, train_dialogues, train_summaries = load_pickle_data(train_dataset_pattern)

In [6]:
print(train_ids[:3])
print(train_ids[-4:-1])

['REPORT-minute-00001-00004', 'REPORT-minute-00001-00006', 'REPORT-minute-00001-00019']
['DACON-1462-7', 'DACON-1462-8', 'DACON-1462-9']


In [7]:
train_dialogues[0]

['수고하셨습니다.   다음은 존경하는 현영희 위원님 인사말씀해 주세요.',
 '저는 부산 출신이고 비례대표 현영희입니다.  우리 교육이 잘 돼야 나라가 잘됩니다. 아이들 미래의 행복을 위해서 열심히 하겠습니다. 그리고 모두 여야를 떠나서 정말 진정성을 가지고 우리 아이를 위한 토의장이 되었으면 하는 바람입니다.  감사합니다. 열심히 하겠습니다.',
 '다음은 김태년 위원님 인사 말씀해 주세요.',
 '반갑습니다. 경기도 성남 수정구 출신 김태년입니다.  4년 만에 상임위원회 회의실에 들어와 봅니다. 17대 때는 주로 경제 파트에서 일을 했었는데요.  교육도 그렇고 과학기술도 그렇고 다 대한민국의 미래와 관련해서 아주 중요한 일들을 한다 이렇게 생각을 합니다. 학부모가 교육비 때문에 고통 받지 않아도 되는 나라 그리고 아이들이 학교 가는 것이 즐거운 그런 대한민국을 만들기 위해서 교육위원으로서 최선을 다해서 노력하겠습니다.  감사합니다.',
 '감사합니다.   다음 박혜자 위원님께서 인사말씀해 주세요.',
 '안녕하세요.   저는 광주 서갑의 박혜자입니다.  사실 제가 대학에 한 22년 정도 근무를 했습니다. 그 인연으로 이 자리에 오게 된 것 같습니다. 저는 사실 초선이기 때문에 여러 가지로 배워야 되는 그러한 위치에 있습니다.  그러나 교과위가 무엇보다도 우리 아이들의 미래를 열어가는 장을 만들어 낼 수 있다라는 점에서 제가 열렬하게 이 교과위에 오기를 기대를 했었고요 또 그러한 제 기대가 맞아 떨어져서 여러분과 함께 하게 된 것 같습니다.   배워 가면서 여러분들과 함께 힘을 합쳐서 정말 우리 아이들의 미래를 만드는 일에 힘을 보태고 싶습니다.  감사합니다.']

In [8]:
train_summaries[:5]

['아이들의 미래를 열어가는 장을 만들 수 있는 교과위에 오기를 기대했던 박 위원은 아이들을 위한 일에 힘을 보태고 싶다고 말했다.',
 '이 위원은 교육과 과학이 국가의 미래이고 서민들의 삶을 결정짓는 중요한 국가적 의제이므로 여야 가리지 않고 위원들과 협조해 성과를 창출하겠다고 했다.',
 '강 위원은 진보 성향의 교육감을 둔 지역은 공교롭게도 2012년 시ㆍ도 교육청 평가에서 매우 미흡 판정을 받았다고 했다.',
 '이 교육과학기술부장관은 주 5일제가 잘 정착하기 위해 학교 차원의 배려가 중요하다고 했다.',
 '김 위원은 야당 국회의원인 시인 도 씨의 작품을 교과서에서 삭제하는 것은 정치적으로 굉장히 민감해지는 사안이라며 장관이 이런 결정에 대한 보고를 못 받을 리 없다고 했다.']

### 2. KR-WordRank 적용

In [9]:
from krwordrank.word import KRWordRank

def extract_keywords(text, min_count=3, max_length=30, verbose=False, beta=0.85, max_iter=30):
    wordrank_extractor = KRWordRank(
        min_count=min_count,  # 단어의 최소 출현 빈도수 (그래프 생성 시)
        max_length=max_length,  # 단어의 최대 길이
        verbose=verbose
    )
    keywords, rank, graph = wordrank_extractor.extract(text, beta, max_iter)

    return keywords

In [10]:
wordrank_extractor = KRWordRank(
    min_count = 3,
    max_length = 30,
    verbose = True
)

beta = 0.85    # PageRank의 decaying factor beta
max_iter = 30

keywords, rank, graph = wordrank_extractor.extract(train_dialogues[0], beta, max_iter)

scan vocabs ... 
num vocabs = 74
done = 26 Early stopped.


In [11]:
all_keywords = []
temp_train_dialogues = train_dialogues[:2000]

for i in range(len(temp_train_dialogues)):
    keywords = extract_keywords(temp_train_dialogues[i])
    all_keywords.append(keywords)

In [12]:
def show_keywords(keywords):
    for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:30]:
        print('%8s:\t%.4f' % (word, r))

In [13]:
show_keywords(all_keywords[1000])

      이번:	2.5126
      하는:	1.9509
      정말:	1.7659
      그런:	1.4616
      있는:	0.9092
      것도:	0.7626
      북한:	0.7387
      우리:	0.7015
      아니:	0.3843


In [14]:
for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:30]:
    print('%8s:\t%.4f' % (word, r))

      해병:	4.2395
    22사단:	2.0997
      대대:	1.6116
      사실:	1.4963
     2사단:	1.2934
      하는:	1.2867
      부분:	1.2535
      곳이:	1.1988
      열악:	0.8311
      경계:	0.8259
      말씀:	0.5411
